# Cartopy and GeoViews

One of Satpy's goals is to make the data it generates useable by other open source python tools. Two of the great plotting options for python developers are Cartopy and GeoViews. In this lesson we'll step through some simple examples to plot data using these tools.

First, let's create our CONUS sector ABI `Scene` and load a band we'd like to look at.

In [ ]:
%run ../init_notebook.py
from glob import glob
from satpy import Scene
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)

In [ ]:
my_channel = 'EDITME'
scn.load([my_channel])

Similar to earlier lessons we use the notebook magic `%matplotlib` command to give us interactive matplotlib plots in our notebook. Overall we will be using features provided by the matplotlib library for plotting, cartopy for coastlines and geographic coordinate handling, and xarray for some utility in plotting our data. The main feature Satpy, or rather Pyresample, brings to the table is the `to_cartopy_crs` method to convert our `AreaDefinition` to a Coordinate Reference System (CRS) object that cartopy can understand.

Once we have that `crs` object we can pass it to our axes creation to tell it the CRS for our plot. We also pass this `crs` to the `imshow` method to tell it what CRS our data is in. This is similar to what we encountered in the resampling lesson. We have a projection that we want (our axes) and the projection our data is in. In this case, they are the same so no resampling has to happen.

We'll also use cartopy's `coastlines` and `gridlines` to add some features to our map.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

crs = scn[my_channel].attrs['area'].to_cartopy_crs()
plt.figure()
ax = plt.axes(projection=crs)

my_data = scn[my_channel]
my_data.plot.imshow(transform=crs)
ax.coastlines()
ax.gridlines()

If we wanted to plot our data on a different projection that what it currently is in, we can have cartopy do some resampling for us. Normally satellite data involves rather large data arrays so it may be better in the long run to use Satpy's resampling methods which may perform better and allow for caching of resampling computations.

Let's plot the same data as before, but this time put it on a Lat/Lon grid (PlateCarree). We'll let cartopy do the resampling for us.

In [ ]:
import cartopy.crs as ccrs
crs = scn[my_channel].attrs['area'].to_cartopy_crs()
plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

my_data = scn[my_channel]
my_data.plot.imshow(transform=crs)
ax.coastlines()
ax.gridlines()

This covers the basics of what can be done with cartopy plotting of Satpy data. Cartopy provides a lot more functionality than shown here and there are many cookbooks and recipes online to make fancier plots.

### Further Research

* [Xarray plotting][1]
* [Cartopy tutorial by Phil Elson][2]
* [Cartopy documentation][3]

  [1]: http://xarray.pydata.org/en/stable/plotting.html
  [2]: https://rabernat.github.io/research_computing_2018/maps-with-cartopy.html
  [3]: https://scitools.org.uk/cartopy/docs/latest/
  

# GeoViews

Another popular tool for plotting is GeoViews by the PyViz team. Satpy provides some simple wrapper methods to easily create GeoViews plots from Satpy's `Scene`. Let's start with some basic plotting of our `scn` object from earlier. We first have to import geoviews and tell it to load the bokeh extension which we'll use to draw the data.

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts
gv.extension("bokeh")
opts.defaults(
    opts.Image(width=600, height=400, colorbar=True),
    opts.Feature(apply_ranges=False),
    opts.QuadMesh(width=600, height=400, colorbar=True),
)

We then need to define our plot region with some notebook magic commands.

To get our data ready for GeoViews we have to use the `Scene` object's `to_geoviews` method and tell it what channel we'd like. We can then use the multiply operator to add features to our plot like coastlines and borders.

In [ ]:
gv_data = scn.to_geoviews(vdims=[my_channel])
gv_data.opts(cmap='RdBu_r')
gv_data * gf.coastline * gf.borders

By default, GeoViews guessed what projection might be best to view our data in. We can also change projection just like we did with cartopy using cartopy CRS objects. Let's view our data in its original projection.

In [ ]:
crs = scn[my_channel].attrs['area'].to_cartopy_crs()
gv_data = scn.to_geoviews(vdims=[my_channel])
gv_data.opts(cmap='RdBu_r', projection=crs)
gv_data * gf.coastline * gf.borders

Or let's try a Lambert Conformal Conic projection:

In [ ]:
import cartopy.crs as ccrs
gv_data = scn.to_geoviews(vdims=[my_channel])
gv_data.opts(cmap='RdBu_r', projection=ccrs.LambertConformal())
gv_data * gf.coastline * gf.borders

## Time Series

Lastly, we can take advantage of GeoViews ability to visualize multiple "frames" of data. We can use this in combination with Satpy's `MultiScene` to easily step through time steps. Let collect files at every 10 minutes for an hour.

In [ ]:
from glob import glob
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_meso/*s201813121[012345]0*.nc')
len(filenames)

In [ ]:
from satpy import MultiScene
mscn = MultiScene.from_files(filenames, reader='abi_l1b')
mscn.load(['C07'])

We've created our MultiScene of multiple frames and loaded the product we'd like to visualize. Our first step to to getting these "frames" in to GeoViews is to use the `MultiScene.blend` method again, but instead of stack images on top of each other like we did in the previous lesson we will build a time series DataArray. This is like the normal 2D dimensional data arrays we've been dealing with, but with an additional `time` dimension.

In [ ]:
from satpy.multiscene import timeseries
ts_scn = mscn.blend(timeseries)
ts_scn['C07']

In [ ]:
ts_gv = ts_scn.to_geoviews(vdims=['C07'], dynamic=True)
ts_gv.opts(cmap='viridis', projection=ccrs.Geostationary(central_longitude=-75.0))
ts_gv * gf.coastline * gf.borders

Thanks to GeoViews we've created an interactive tool for analyzing a series of satellite images in about 10 lines of code.